# Задание 1 (2 балла)

Напишите класс `MyDict`, который будет полностью повторять поведение обычного словаря, за исключением того, что при итерации мы должны получать и ключи, и значения.

**Модули использовать нельзя**

In [1]:
class MyDict(dict):
    '''A derivative class of a regular dictionary, with the exception of iteration giving both keys and values'''
    
    
    def __iter__(self):
        self.iterator = iter(self.items())
        return self
    
    
    def __next__(self):
        return next(self.iterator)

In [2]:
dct = MyDict({"a": 1, "b": 2, "c": 3, "d": 25})
for key, value in dct:
    print(key, value)   

a 1
b 2
c 3
d 25


In [3]:
for key, value in dct.items():
    print(key, value)

a 1
b 2
c 3
d 25


In [4]:
for key in dct.keys():
    print(key)

a
b
c
d


In [5]:
dct["c"] + dct["d"]

28

# Задание 2 (2 балла)

Напишите функцию `iter_append`, которая "добавляет" новый элемент в конец итератора, возвращая итератор, который включает изначальные элементы и новый элемент. Итерироваться по итератору внутри функции нельзя, то есть вот такая штука не принимается
```python
def iter_append(iterator, item):
    lst = list(iterator) + [item]
    return iter(lst)
```

**Модули использовать нельзя**

In [6]:
def iter_append(iterator, item):
    '''Apppends new element to the list iterator
    
    Parameters:
        iterator (list_iterator): list iterator
        item: any python object
    Yields: generator
    '''
    
    
    yield from iterator
    yield item

In [7]:
my_iterator = iter([1, 2, 3])
new_iterator = iter_append(my_iterator, 4)

for element in new_iterator:
    print(element)

1
2
3
4


In [8]:
# checking type
type(new_iterator)

generator

# Задание 3 (5 баллов)

Представим, что мы установили себе некотурую библиотеку, которая содержит в себе два класса `MyString` и `MySet`, которые являются наследниками `str` и `set`, но также несут и дополнительные методы.

Проблема заключается в том, что библиотеку писали не очень аккуратные люди, поэтому получилось так, что некоторые методы возвращают не тот тип данных, который мы ожидаем. Например, `MyString().reverse()` возвращает объект класса `str`, хотя логичнее было бы ожидать объект класса `MyString`.

Найдите и реализуйте удобный способ сделать так, чтобы подобные методы возвращали экземпляр текущего класса, а не родительского. При этом **код методов изменять нельзя**

**+3 дополнительных балла** за реализацию того, чтобы **унаследованные от `str` и `set` методы** также возвращали объект интересующего нас класса (то есть `MyString.replace(..., ...)` должен возвращать `MyString`). **Переопределять методы нельзя**

**Модули использовать нельзя**

In [9]:
def type_for_methods(cls):
    '''Searches for public class methods and sets type for their output
    
    Parametes:
        cls: class
    Returns: class with new type for class methods outputs
    '''
    
    
    for attr in dir(cls):
        if not attr.startswith('_') and callable(getattr(cls, attr)):
            setattr(cls, attr, change_type(getattr(cls, attr), cls))
    return cls


def change_type(func, cls):
    '''Function change_type() is a decorator for class and changes type of class method output 
    with MyString for strings and MySet for sets
    
    Parameters:
        func: class method
        cls: class
    Returns: inner_function
    '''
    
    
    def inner_function(*args, **kwargs):
        '''Determines the type of class method output and sets appropriate one
        
        Parameters:
            *args
            **kwargs
        Returns: result (MyString or MySet object)
        '''
        
        
        result = func(*args, **kwargs)
        if (cls == MyString and isinstance(result, str)) or ( # isinstance to strict str or set type output
            cls == MySet and isinstance(result, set)):
            return cls(result)
        else:
            return result
    return inner_function


@type_for_methods
class MyString(str):
    def reverse(self):
        return self.__class__(self[::-1])
    
    def make_uppercase(self):
        return "".join([chr(ord(char) - 32) if 97 <= ord(char) <= 122 else char for char in self])
    
    def make_lowercase(self):
        return "".join([chr(ord(char) + 32) if 65 <= ord(char) <= 90 else char for char in self])
    
    def capitalize_words(self):
        return " ".join([word.capitalize() for word in self.split()])

    
@type_for_methods    
class MySet(set):
    def is_empty(self):
        return len(self) == 0
    
    def has_duplicates(self):
        return len(self) != len(set(self))
    
    def union_with(self, other):
        return self.union(other)
    
    def intersection_with(self, other):
        return self.intersection(other)
    
    def difference_with(self, other):
        return self.difference(other)

In [10]:
string_example = MyString("Aa Bb Cc")
set_example_1 = MySet({1, 2, 3, 4})
set_example_2 = MySet({3, 4, 5, 6, 6})

print(type(string_example.reverse()))
print(type(string_example.make_uppercase()))
print(type(string_example.make_lowercase()))
print(type(string_example.capitalize_words()))
print()
print(type(set_example_1.is_empty()))
print(type(set_example_2.has_duplicates()))
print(type(set_example_1.union_with(set_example_2)))
print(type(set_example_1.difference_with(set_example_2)))

<class '__main__.MyString'>
<class '__main__.MyString'>
<class '__main__.MyString'>
<class '__main__.MyString'>

<class 'bool'>
<class 'bool'>
<class '__main__.MySet'>
<class '__main__.MySet'>


In [11]:
print(string_example.replace("A", "D"))
print(type(string_example.replace("A", "D")))

Da Bb Cc
<class '__main__.MyString'>


# Задание 4 (5 баллов)

Напишите декоратор `switch_privacy`:
1. Делает все публичные **методы** класса приватными
2. Делает все приватные методы класса публичными
3. Dunder методы и защищённые методы остаются без изменений
4. Должен работать тестовый код ниже, в теле класса писать код нельзя

**Модули использовать нельзя**

In [12]:
def switch_privacy(cls):
    '''Switches privacy for class methods: 
    makes public methods to be private, and vice versa
    
    Parameters:
        cls: class
    Returns: class with switched public and private methods
    '''
    
    
    copy_attr_dict = cls.__dict__.copy()
    for attr in copy_attr_dict:
        if callable(getattr(cls, attr)):
            if not attr.startswith('_'):
                # private methods looks like "_ExampleClass__private_method" in dir
                setattr(cls, f'_{cls.__name__}__{attr}', getattr(cls, attr))
                delattr(cls, attr)
            elif attr.startswith(f'_{cls.__name__}__') and not attr.endswith('__'):
                setattr(cls, attr.split('__')[1], getattr(cls, attr))
                delattr(cls, attr)
    return cls

In [13]:
@switch_privacy
class ExampleClass:
    # Но не здесь
    def public_method(self):
        return 1
    
    def _protected_method(self):
        return 2
    
    def __private_method(self):
        return 3
    
    def __dunder_method__(self):
        pass

In [14]:
test_object = ExampleClass()

test_object._ExampleClass__public_method()   # Публичный метод стал приватным

1

In [15]:
test_object.private_method()   # Приватный метод стал публичным

3

In [16]:
test_object._protected_method()   # Защищённый метод остался защищённым

2

In [17]:
test_object.__dunder_method__()   # Дандер метод не изменился

In [18]:
hasattr(test_object, "public_method"), hasattr(test_object, "__private_method")   # Изначальные варианты изменённых методов не сохраняются

(False, False)

# Задание 5 (7 баллов)

Напишите [контекстный менеджер](https://docs.python.org/3/library/stdtypes.html#context-manager-types) `OpenFasta`

Контекстные менеджеры это специальные объекты, которые могут работать с конструкцией `with ... as ...:`. В них нет ничего сложного, для их реализации как обычно нужно только определить только пару dunder методов. Изучите этот вопрос самостоятельно

1. Объект должен работать как обычные файлы в питоне (наследоваться не надо, здесь лучше будет использовать **композицию**), но:
    + При итерации по объекту мы должны будем получать не строку из файла, а специальный объект `FastaRecord`. Он будет хранить в себе информацию о последовательности. Важно, **не строки, а именно последовательности**, в fasta файлах последовательность часто разбивают на много строк
    + Нужно написать методы `read_record` и `read_records`, которые по смыслу соответствуют `readline()` и `readlines()` в обычных файлах, но они должны выдавать не строки, а объект(ы) `FastaRecord`
2. Конструктор должен принимать один аргумент - **путь к файлу**
3. Класс должен эффективно распоряжаться памятью, с расчётом на работу с очень большими файлами
    
Объект `FastaRecord`. Это должен быть **датакласс** (см. про примеры декораторов в соответствующей лекции) с тремя полями:
+ `seq` - последовательность
+ `id_` - ID последовательности (это то, что в фаста файле в строке, которая начинается с `>` до первого пробела. Например, >**GTD326487.1** Species anonymous 24 chromosome) 
+ `description` - то, что осталось после ID (Например, >GTD326487.1 **Species anonymous 24 chromosome**)


Напишите демонстрацию работы кода с использованием всех написанных методов, обязательно добавьте файл с тестовыми данными в репозиторий (не обязательно большой)

**Можно использовать модули из стандартной библиотеки**

In [19]:
import os
from dataclasses import dataclass


class OpenFasta:
    '''A class for reading .fasta files with biological sequences
    
    Record example: >Seq_ID Description(optional)
                    ATGTGGGAGGAAGGTGGGTGAAA
             
    Attributes:
        path (str): path to .fasta file
        
    Methods:
        read_record(): to read one record from .fasta file
        read_records(): to read multiple records from .fasta file
        
    Raises: ValueError if .fasta file does not exist in path        
    '''
    
    
    def __init__(self, path):
        '''Checks the existance of .fasta file and constructs all the necessary attributes

        Parameters:
            path (str): path to .fasta file
        
        Raises: ValueError if .fasta file does not exist in path
        '''
        
        
        if not os.path.isfile(path):
            file_path = path.split(os.path.sep)
            raise ValueError(f'No such {file_path[-1]}. Check path to file!')
    
        self.path = path
        
    
    def __enter__(self):
        self.f = open(self.path)
        return self
    
    
    def __iter__(self):
        return self
    
    
    def __next__(self):
        self.seq = []
        while True: 
            line = self.f.readline()
            if not line:
                if self.seq:
                    self.seq = ''.join(self.seq)
                    record = FastaRecord(id_=self.id_,
                                         description=self.description,
                                         seq=self.seq)
                    return record
                return ''
            elif line.startswith('>'):
                if self.seq:
                    self.seq = ''.join(self.seq)
                    record = FastaRecord(id_=self.id_,
                                         description=self.description,
                                         seq=self.seq)
                    self.id_, self.description = self.__handle_id_descr(line)
                    return record
                self.id_, self.description = self.__handle_id_descr(line)
            else:
                self.seq.append(line.strip())
                
                
    def __exit__(self, exc_type, exc_value, exc_tb):
        self.f.close()
        pass
               
            
    def __handle_id_descr(self, line):
        '''Handles the first record line with Seq_ID and Description (optional)
        
        Parameters:
            line (str): record line
        Returns:
            id_ (str): record Seq_ID
            description (str): record description (if exists)
        '''
        
        
        vals = line.strip().split(' ', maxsplit=1)
        id_ = vals[0][1:]
        if len(vals) == 1:
            description = ''
        else: 
            description = vals[1]
        return id_, description
            
                
    def read_record(self):
        '''Reads fasta record
        
        Returns: FastaRecord
        '''
        
        
        return next(self)
    
    
    def read_records(self):
        '''Reads all fasta records in file and appends them to list
        
        Returns: records (list)
        '''
        
        
        records = []
        record = self.read_record()
        while record:
            records.append(record)
            record = self.read_record()
        return records
              

@dataclass
class FastaRecord:
    '''A dataclass with information about fasta record
    
    Parameters:
        id_ (str): record id (Seq_ID)
        description (str): record description (if exists)
        seq (str): biological sequences
    '''
    
    
    id_: str
    description: str
    seq: str

#### Example1. Reading one fasta record from file

In [20]:
with OpenFasta(os.path.join("data", "example.fasta")) as fasta:
    record = fasta.read_record()
    print(f'FastaRecord form file:\n{record}')
    print(f'Seq_ID: {record.id_}\nSeq: {record.seq}')
    print(type(record))

FastaRecord form file:
FastaRecord(id_='SUP35_Kla_AB039749', description='', seq='ATGTCAGACCAACAAAATCAAGACCAAGGGCAAGGCCAAGGTTACAATCAGTATAACCAATATGGCCAGTACAACCAGTACTACAACCAACAGGGCTATCAAGGCTACAACGGCCAACAAGGTGCTCCTCAAGGCTACCAAGCATATCAAGCTTATGGACAGCAGCCTCAAGGAGCCTACCAGGGCTACAACCCTCAACAAGCTCAAGGC')
Seq_ID: SUP35_Kla_AB039749
Seq: ATGTCAGACCAACAAAATCAAGACCAAGGGCAAGGCCAAGGTTACAATCAGTATAACCAATATGGCCAGTACAACCAGTACTACAACCAACAGGGCTATCAAGGCTACAACGGCCAACAAGGTGCTCCTCAAGGCTACCAAGCATATCAAGCTTATGGACAGCAGCCTCAAGGAGCCTACCAGGGCTACAACCCTCAACAAGCTCAAGGC
<class '__main__.FastaRecord'>


#### Example2. Reading all fasta records from file

In [21]:
with OpenFasta(os.path.join("data", "example.fasta")) as fasta:
    print(f'All FastaRecords from file:')
    for record in fasta.read_records():
        print(record)

All FastaRecords from file:
FastaRecord(id_='SUP35_Kla_AB039749', description='', seq='ATGTCAGACCAACAAAATCAAGACCAAGGGCAAGGCCAAGGTTACAATCAGTATAACCAATATGGCCAGTACAACCAGTACTACAACCAACAGGGCTATCAAGGCTACAACGGCCAACAAGGTGCTCCTCAAGGCTACCAAGCATATCAAGCTTATGGACAGCAGCCTCAAGGAGCCTACCAGGGCTACAACCCTCAACAAGCTCAAGGC')
FastaRecord(id_='SUP35_Agos_ATCC_10895_NM_211584', description='', seq='ATGTCGGAGGAAGATCAAATTCAATCGCAAGGCAACGACCAAGGCCAGTCGCAAGCCAAGGATCAAGGTCAAAATCAAGGTCAGGGGCAGCAAAATTTCGGCCAATACTACAACCCAAGTAACTTCCAGAATTACCAGGG')
FastaRecord(id_='SUP35_Scer_74-D694_GCA_001578265.1', description='SUP35 gene, Saccharomyces cerevisiae', seq='ATGTCGGATTCAAACCAAGGCAACAATCAGCAAAACTACCAGCAATACAGCCAGAACGGTAACCAACAACAAGGTAACAACAGATACCAAGGTTATCAAGCTTACAATGCTCAAGCCCAACCTGCAGGTGGGTACTACCAAAATTACCAAGGTTATTCTGGGTACCAACAAGGTGGCTATCAACAGTACAATCCCGACGCCGGTTACCAG')


#### Example3. At the end of the file acts like `readlines()`

In [22]:
with OpenFasta(os.path.join("data", "example.fasta")) as fasta:
    for _ in range(10):
        print(fasta.read_record())    

FastaRecord(id_='SUP35_Kla_AB039749', description='', seq='ATGTCAGACCAACAAAATCAAGACCAAGGGCAAGGCCAAGGTTACAATCAGTATAACCAATATGGCCAGTACAACCAGTACTACAACCAACAGGGCTATCAAGGCTACAACGGCCAACAAGGTGCTCCTCAAGGCTACCAAGCATATCAAGCTTATGGACAGCAGCCTCAAGGAGCCTACCAGGGCTACAACCCTCAACAAGCTCAAGGC')
FastaRecord(id_='SUP35_Agos_ATCC_10895_NM_211584', description='', seq='ATGTCGGAGGAAGATCAAATTCAATCGCAAGGCAACGACCAAGGCCAGTCGCAAGCCAAGGATCAAGGTCAAAATCAAGGTCAGGGGCAGCAAAATTTCGGCCAATACTACAACCCAAGTAACTTCCAGAATTACCAGGG')
FastaRecord(id_='SUP35_Scer_74-D694_GCA_001578265.1', description='SUP35 gene, Saccharomyces cerevisiae', seq='ATGTCGGATTCAAACCAAGGCAACAATCAGCAAAACTACCAGCAATACAGCCAGAACGGTAACCAACAACAAGGTAACAACAGATACCAAGGTTATCAAGCTTACAATGCTCAAGCCCAACCTGCAGGTGGGTACTACCAAAATTACCAAGGTTATTCTGGGTACCAACAAGGTGGCTATCAACAGTACAATCCCGACGCCGGTTACCAG')









# Задание 6 (7 баллов)

1. Напишите код, который позволит получать все возможные (неуникальные) генотипы при скрещивании двух организмов. Это может быть функция или класс, что вам кажется более удобным.

Например, все возможные исходы скрещивания "Aabb" и "Aabb" (неуникальные) это

```
AAbb
AAbb
AAbb
AAbb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
aabb
aabb
aabb
aabb
```

2. Напишите функцию, которая вычисляет вероятность появления определённого генотипа (его ожидаемую долю в потомстве).
Например,

```python
get_offspting_genotype_probability(parent1="Aabb", parent2="Aabb", target_genotype="Аabb")   # 0.5

```

3. Напишите код, который выводит все уникальные генотипы при скрещивании `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` и `'АаббВвГгДДЕеЖжЗзИиЙйКкЛлМмНН'`, которые содержат в себе следующую комбинацию аллелей `'АаБбВвГгДдЕеЖжЗзИиЙйКкЛл'`
4. Напишите код, который расчитывает вероятность появления генотипа `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` при скрещивании `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн` и `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн`

Важные замечания:
1. Порядок следования аллелей в случае гетерозигот всегда должен быть следующим: сначала большая буква, затем маленькая (вариант `AaBb` допустим, но `aAbB` быть не должно)
2. Подзадачи 3 и 4 могут потребовать много вычислительного времени (до 15+ минут в зависимости от железа), поэтому убедитесь, что вы хорошо протестировали написанный вами код на малых данных перед выполнением этих задач. Если ваш код работает **дольше 20 мин**, то скорее всего ваше решение не оптимально, попытайтесь что-нибудь оптимизировать. Если оптимальное решение совсем не получается, то попробуйте из входных данных во всех заданиях убрать последний ген (это должно уменьшить время выполнения примерно в 4 раза), но **за такое решение будет снято 2 балла**
3. Несмотря на то, что подзадания 2, 3 и 4 возможно решить математически, не прибегая к непосредственному получению всех возможных генотипов, от вас требуется именно brute-force вариант алгоритма

**Можно использовать модули из стандартной библиотеки питона**, но **за выполнение задания без использования модулей придусмотрено +3 дополнительных балла**

In [23]:
from functools import lru_cache
from datetime import datetime


# Ваш код здесь (1 и 2 подзадание)
def calc_runtime(func):
    '''Decorator for calculatinf function runtime'''
    
    
    def inner_function(*args, **kwargs):
        start = datetime.now()
        result = func(*args, **kwargs)
        stop = datetime.now()
        print(stop - start)
        return result
    return inner_function  


def parse_genotype(genotype):
    '''Parses genotypes to obtain alleles
    
    Parameters:
        genotype (str): genotype, example: 'Aabb'
    
    Returns: list of alleles
    '''
    
    
    return [genotype[i:i+2] for i in range(0, len(genotype), 2)]


def calc_gamets(*args):
    '''Creates sets with all possible gamets for genotype
    
    Parameters:
        *args (alleles returned by parse_genotype())
    Returns: generator with all possible gamets for genotype
    '''
    
    
    pools = [tuple(pool) for pool in args]
    result = [[]]
    for pool in pools:
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)
        

@lru_cache(maxsize=None)
def do_genotype(*gamets):
    '''Makes genotype from gamets
    
    Parameters:
        gamets (tuples): two sets of gamets
    Returns: 
        genotype (str): resulted genotype
    '''
    
    
    genotype = ''.join([''.join(sorted(gene)) for gene in zip(*gamets)])
    return genotype

        
def calc_genotypes(gamets1, gamets2, k=1):
    '''Creates all possible genotypes from two sets of gamets
    
    Parameters:
        gamets1 (tuple): set of gamets for the first genotype
        gamets2 (tuple): set of gamets for the second genotype
        k (int): number of genes devided by two
    
    Returns: generator with possible genotypes
    '''
    
    
    for gamet1 in gamets1:
        for gamet2 in gamets2:
            geno_l = do_genotype(gamet1[:k], gamet2[:k])
            geno_r = do_genotype(gamet1[k:], gamet2[k:])
            yield geno_l + geno_r   

            
def all_genotypes(p1, p2):
    '''Creates all possible offspring genotypes for given parental genotypes
    
    Parameters:
        p1 (str): parent1 genotype
        p2 (str): parent2 genotype
        
    Returns: None
    '''
    
    
    gamets1 = tuple(calc_gamets(*parse_genotype(p1)))
    gamets2 = tuple(calc_gamets(*parse_genotype(p2)))
    [print(genotype) for genotype in calc_genotypes(gamets1, gamets2)]
    

@calc_runtime
def genotype_prob(p1, p2, target):
    '''Calculates the probability of a certain genotype (its expected share in the offspring)
    
    Parameters:
        p1 (str): parent1 genotype
        p2 (str): parent2 genotype
        target (str): target offspring genotype for its probability calculation
        
    Returns: float (probability)
    '''
    
    
    gamets1 = tuple(calc_gamets(*parse_genotype(p1)))
    gamets2 = tuple(calc_gamets(*parse_genotype(p2)))
    
    total_count = target_count = 0
    k = len(p1) // 4
    genotypes = calc_genotypes(gamets1, gamets2, k)
    for genotype in genotypes:
        total_count += 1
        if target == genotype:
            target_count +=1
    return target_count/total_count

In [24]:
# example for Subtask1
parent1 = 'Aabb'
parent2 = 'Aabb'
all_genotypes(parent1, parent2)

AAbb
AAbb
Aabb
Aabb
AAbb
AAbb
Aabb
Aabb
Aabb
Aabb
aabb
aabb
Aabb
Aabb
aabb
aabb


In [25]:
# example for Subtask2
parent1 = 'Aabb'
parent2 = 'Aabb'
target = 'Aabb'
genotype_prob(parent1, parent2, target)

0:00:00.000024


0.5

In [26]:
# Ваш код здесь (3 подзадание)
@calc_runtime
def unique_with(p1, p2, target):
    '''Finds all unique genotypes with certain alleles combination
    
    Parameters:
        p1 (str): parent1 genotype
        p2 (str): parent2 genotype
        target (str): target offspring genotype
        
    Returns: None
    '''
    
    
    gamets1 = tuple(calc_gamets(*parse_genotype(p1)))
    gamets2 = tuple(calc_gamets(*parse_genotype(p2)))
    
    unique = []
    for genotype in calc_genotypes(gamets1, gamets2):
        if (genotype.find(target) != -1) and (genotype not in unique):
            unique += [genotype]
            print(genotype)

In [27]:
# example for Subtask3
parent1 = 'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'
parent2 = 'АаббВвГгДДЕеЖжЗзИиЙйКкЛлМмНН'
target = 'АаБбВвГгДдЕеЖжЗзИиЙйКкЛл'
unique_with(parent1, parent2, target)

АаБбВвГгДдЕеЖжЗзИиЙйКкЛлММНН
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНН
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлММНн
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлммНН
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлммНн
0:02:15.341380


In [28]:
# Ваш код здесь (4 подзадание)
# example for Subtask4
parent1 = 'АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн'
parent2 = 'АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн'
target = 'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'
genotype_prob(parent1, parent2, target)

0:01:47.510493


1.52587890625e-05